- In Many application data may be spread in different files and folders. Due to this analyze of data will typical so we focus on data Wargling which main purpose of data join, combine, and reshape

# Hierarchical Indexing

In [1]:
import pandas as pd
import numpy as np
# allow working with multi level index means multideminsinal data into low dimensional data

In [2]:
data = pd.Series(np.random.uniform(size=9), index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"], [1,2,3,1,3,1,2,2,3]])

In [3]:
data

a  1    0.948857
   2    0.939386
   3    0.593083
b  1    0.541384
   3    0.665999
c  1    0.069711
   2    0.087997
d  2    0.558306
   3    0.258390
dtype: float64

In [4]:
data["a"]

1    0.948857
2    0.939386
3    0.593083
dtype: float64

In [5]:
data["a":"c"]

a  1    0.948857
   2    0.939386
   3    0.593083
b  1    0.541384
   3    0.665999
c  1    0.069711
   2    0.087997
dtype: float64

In [6]:
data.loc[:,2]

a    0.939386
c    0.087997
d    0.558306
dtype: float64

In [7]:
data.unstack()

,1,2,3
a,0.948857,0.939386,0.593083
b,0.541384,NaN,0.665999
c,0.069711,0.087997,NaN
d,NaN,0.558306,0.258390


In [8]:
data.unstack().stack()

a  1    0.948857
   2    0.939386
   3    0.593083
b  1    0.541384
   3    0.665999
c  1    0.069711
   2    0.087997
d  2    0.558306
   3    0.258390
dtype: float64

In [9]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),index=[["a", "a", "b", "b"], [1, 2, 1, 2]],columns=[["Ohio", "Ohio", "Colorado"],["Green", "Red", "Green"]])

In [10]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
frame.index.names=["key1", "key2"]

In [12]:
frame.columns.names=["state", "color"]

In [13]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [14]:
frame.index.nlevels

2

# reodering and Sorting values

In [15]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [16]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [17]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [18]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [19]:
#Summary Statistics by level
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [20]:
frame.groupby(level="color", axis="columns").sum()

/tmp/ipykernel_2295/775557097.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color", axis="columns").sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

# Combining and Merging datasets

1. pd.merge:- Connect row in dataframes based on one or more keys
2. pd.concat:- Concatinate or "stack" objects together along an axis
3. combine_first:- 

In [21]:
#Many to one
df1 = pd.DataFrame({"key":["b", "a", "a", "c", "b", "c"], "data1":pd.Series(range(6), dtype="Int64")})

In [22]:
df2 = pd.DataFrame({"key":["b", "a", "d"], "data2":pd.Series(range(3), dtype="Int64")})

In [23]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,0
1,a,1,1
2,a,2,1
3,b,4,0


In [24]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,0
1,a,1,1
2,a,2,1
3,b,4,0


In [25]:
#If the columns name are different so we also do specify
df3 = pd.DataFrame({"lkey":["b", "a", "a", "c", "b", "c"], "data1":pd.Series(range(6), dtype="Int64")})
df4 = pd.DataFrame({"rkey":["b", "a", "d"], "data2":pd.Series(range(3), dtype="Int64")})

In [26]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,0
1,a,1,a,1
2,a,2,a,1
3,b,4,b,0


- You see that d and c are missing because by default pd.merge take key intersection
- Other possible option are present like left, right, and outer. Outer take a union of the key

In [27]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,1,1
1,a,2,1
2,b,0,0
3,b,4,0
4,c,3,<NA>
5,c,5,<NA>
6,d,<NA>,2


In [28]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,1,a,1
1,a,2,a,1
2,b,0,b,0
3,b,4,b,0
4,c,3,NaN,<NA>
5,c,5,NaN,<NA>
6,NaN,<NA>,d,2


In [29]:
#Many to Many
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],"data1": pd.Series(range(6), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],"data2": pd.Series(range(5), dtype="Int64")})

In [30]:
pd.merge(df1, df2, how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [31]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],"key2": ["one", "two", "one"],"lval": pd.Series([1, 2, 3], dtype='Int64')})
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],"key2": ["one", "one", "one", "two"],"rval": pd.Series([4, 5, 6, 7], dtype='Int64')})

In [32]:
#multiple value merging
pd.merge(left, right, on=["key1"], how="outer", suffixes=("_left","_right"))

,key1,key2_left,lval,key2_right,rval
0,bar,one,3,one,6
1,bar,one,3,two,7
2,foo,one,1,one,4
3,foo,one,1,one,5
4,foo,two,2,one,4
5,foo,two,2,one,5


In [33]:
#When you join the columns so the index of data discard so you need to use rest_index 
#there are also more present pandas.merge function arguments

In [34]:
#some time you find merge key in index so we use left_index=True and right_index=True

In [35]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],"value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [36]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [38]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],index=["a", "c", "e"],columns=["Ohio", "Nevada"]).astype("Int64")
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],index=["b", "c", "d", "e"],columns=["Missouri", "Alabama"]).astype("Int64")

In [39]:
#join method is also consist to simplfy merging by index
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [40]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],index=["a", "c", "e", "f"],columns=["New York", "Oregon"])

In [41]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


# Concatination along axis

In [42]:
arr = np.arange(12).reshape(4,3)

In [44]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  0,  1,  2],
       [ 3,  4,  5,  3,  4,  5],
       [ 6,  7,  8,  6,  7,  8],
       [ 9, 10, 11,  9, 10, 11]])

In [46]:
s1 = pd.Series([0, 1], index=['a', 'b'], dtype='Int64')
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'], dtype="Int64")
s3 = pd.Series([5, 6, 7], index=['f', 'g', 'h'], dtype="Int64")

In [47]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
h    7
dtype: Int64

In [50]:
pd.concat([s1, s2], axis='columns')

,0,1
a,0,<NA>
b,1,<NA>
c,<NA>,2
d,<NA>,3
e,<NA>,4


In [54]:
pd.concat([s1, s2, s3], keys=["one", "two", "three"], axis="columns").unstack()

one    a       0
       b       1
       c    <NA>
       d    <NA>
       e    <NA>
       f    <NA>
       g    <NA>
       h    <NA>
two    a    <NA>
       b    <NA>
       c       2
       d       3
       e       4
       f    <NA>
       g    <NA>
       h    <NA>
three  a    <NA>
       b    <NA>
       c    <NA>
       d    <NA>
       e    <NA>
       f       5
       g       6
       h       7
dtype: Int64

In [55]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],columns=["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],columns=["three", "four"])

In [56]:
pd.concat([df1, df2], axis="columns")

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [59]:
pd.concat({"level1":df1, "level2":df2}, axis="columns", names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [60]:
#we can also combine the data by overlaping

In [61]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],index=["f", "e", "d", "c", "b", "a"])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],index=["a", "b", "c", "d", "e", "f"])

In [62]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

# Reshaping and pivoting

In [63]:
#Reshaping = stack and unstack
# pivotion = A common way to store a multiple time series in databases and csv file is called long and stacked format